In [19]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [20]:
dfo = pd.read_excel('data.xlsx')


In [21]:
dfo.columns

Index(['SLET2T_ID', 'Surname', 'gender', 'Age_at_inclusion',
       'SLE_duration_at_inclusion_months', 'case_no', 'Visit_no',
       'Cumulative_prednisolone_intake_daily', 'PhGA', 'S2K_item_1',
       'S2K_item_2', 'S2K_item_3', 'S2K_item_4', 'S2K_item_5', 'S2K_item_6',
       'S2K_item_7', 'S2K_item_8', 'S2K_item_9', 'S2K_item_10', 'S2K_item_11',
       'S2K_item_12', 'S2K_item_13', 'S2K_item_14', 'S2K_item_15',
       'S2K_item_16', 'S2K_item_17', 'S2K_item_18', 'S2K_item_19',
       'S2K_item_20', 'S2K_item_21', 'S2K_item_22', 'S2K_item_23',
       'S2K_item_24', 'SLEDAI2K', 'Clinical_S2K', 'SFI',
       'SFI_excluding_inclusion_visit', 'Severe_SFI_excluding_inclusion_visit',
       'Severe_SFI_occuring_at_next_visit_new', 'SDI',
       'New_SDI_since_previous_visit_new', 'S2K_item9_extra',
       'S2K_item12_extra', 'S2K_item13_extra', 'S2K_item14_extra',
       'S2K_item15_extra', 'S2K_item16_extra', 'S2K_item23_extra',
       'S2K_item24_extra', 'Visit_month_new', 'LLDAS', 'DOR

In [22]:
filtered_s2k = [col for col in dfo.columns if 'S2K_item_' in col][:24]
# filtered_sdi = [col for col in dfo.columns if 'New_SDI_since_previous_visit' in col]

filtered_columns = []

filtered_columns.append(dfo.SLET2T_ID.name)
filtered_columns.append(dfo.Visit_month_new.name)
filtered_columns.extend(filtered_s2k)
filtered_columns.append(dfo.SDI.name)
filtered_columns.append(dfo.New_SDI_since_previous_visit_new.name)


In [23]:
df = dfo[filtered_columns]


In [24]:
df[df.SLET2T_ID == 'CRE11066']

,SLET2T_ID,Visit_month_new,S2K_item_1,S2K_item_2,S2K_item_3,S2K_item_4,S2K_item_5,S2K_item_6,S2K_item_7,S2K_item_8,S2K_item_9,S2K_item_10,S2K_item_11,S2K_item_12,S2K_item_13,S2K_item_14,S2K_item_15,S2K_item_16,S2K_item_17,S2K_item_18,S2K_item_19,S2K_item_20,S2K_item_21,S2K_item_22,S2K_item_23,S2K_item_24,SDI,New_SDI_since_previous_visit_new
0,CRE11066,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0.0,0.0
1,CRE11066,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0.0,0.0
2,CRE11066,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0.0,0.0
3,CRE11066,18,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0.0,0.0


In [27]:
max_visits = df['Visit_month_new'].max()

# Create sequences
sequences = []
for case_no, group in df.groupby('SLET2T_ID'):
    # Reindex to include all visit_no from 1 to max_visits
    group = group.set_index('Visit_month_new').reindex(range(1, max_visits + 1), fill_value=0)
    sequences.append(group.drop('SLET2T_ID', axis=1).values)

# Pad sequences to the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequences = pad_sequences(sequences, padding='post', dtype='float32')

# Separate features and labels (assuming SDI is the target for the next visit)
X = padded_sequences[:, :, :-2]  # All columns except 'SDI' and 'New_SDI_since_previous_visit_new'
y = padded_sequences[:, :, -2]   # SDI (assuming SDI is the second last column)




In [28]:
X[0].shape

(60, 24)

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking

# Define model
model = Sequential()

# Add masking layer to handle variable sequence lengths
# model.add(Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])))

# Add LSTM layer
model.add(LSTM(64, return_sequences=True))

# Add Dense layer
model.add(Dense(1))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy','precision','recall'])

# Summary of the model
model.summary()

from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9305 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.9474 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9409 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.9474 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9436 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.9474 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9369 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.9474 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9429 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+